In [22]:
# Import packages and set numpy random seed
import numpy as np
np.random.seed(5) 
import cv2

from tensorflow.keras.preprocessing.image import ImageDataGenerator


import matplotlib.pyplot as plt
%matplotlib inline

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    '../../ddd_images_test/all',
    target_size=(128, 128),
    batch_size=16,
    shuffle=True,
    class_mode='binary')


Found 173254 images belonging to 2 classes.


In [23]:
#for GPU training
import tensorflow as tf
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


config = tf.compat.v1.ConfigProto(gpu_options = 
                         tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.33)
# device_count = {'GPU': 1}
)
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 538050346434174120
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 3173331764
locality {
  bus_id: 1
  links {
  }
}
incarnation: 198285356454358334
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 960M, pci bus id: 0000:02:00.0, compute capability: 5.0"
]


In [24]:
import keras
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Flatten, Dense, Dropout, BatchNormalization, Reshape, Permute, Multiply, Input, Activation
from keras.models import Sequential, Model

model = Sequential(
[
    keras.Input((128,128,3)),
    Conv2D(32, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(32, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    Dropout(0.5),

    Conv2D(16, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Conv2D(8, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),
    Dropout(0.5),

    Conv2D(4, 5, activation='relu', padding = 'same'),
    BatchNormalization(),
    MaxPooling2D((2,2)),

    Flatten(),
    Dense(1, activation='sigmoid')
]
)
optimizer = keras.optimizers.Adam()
loss_fn = keras.losses.BinaryCrossentropy(from_logits=True) #from_logits=True means output probabilities are not normalized
acc_metric = keras.metrics.BinaryAccuracy()
val_acc_metric = keras.metrics.BinaryAccuracy()

model.summary()
model.load_weights('data_aug_off_model.h5')

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_33 (Conv2D)           (None, 128, 128, 32)      2432      
_________________________________________________________________
batch_normalization_32 (Batc (None, 128, 128, 32)      128       
_________________________________________________________________
max_pooling2d_31 (MaxPooling (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 64, 64, 32)        25632     
_________________________________________________________________
batch_normalization_33 (Batc (None, 64, 64, 32)        128       
_________________________________________________________________
max_pooling2d_32 (MaxPooling (None, 32, 32, 32)        0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 32, 32, 32)       

In [25]:
@tf.function
def test_step(x, y):
    val_preds = model(x, training=False)
    loss = val_loss_fn(y, val_preds)
    
    # Update val metrics
    val_acc_metric.update_state(y, val_preds)
    return val_preds, loss

In [26]:
valid_writer = tf.summary.create_file_writer('logs/base_aug_off_test/test', max_queue = 10)

total_valid_files = 0

for val_batch_idx in range(test_generator.samples//16):
    recent_test_batch = test_generator.next()
    x_test_batch = recent_test_batch[0]
    y_test_batch = recent_test_batch[1]

    val_y_preds,val_loss = test_step(x_test_batch,y_test_batch)
    val_y_pred = np.reshape(tf.get_static_value(val_y_preds),(1,16))[0]
    test_accuracy = val_acc_metric.result()
    if val_batch_idx%16 == 0 and val_batch_idx!=0:
        with valid_writer.as_default():
            tf.summary.scalar("test_accuracy", test_accuracy, step = total_valid_files)
        print("Test Accuracy for batch {} is: ".format(val_batch_idx), test_accuracy)
        total_valid_files += 16

Test Accuracy for batch 16 is:  tf.Tensor(0.5992647, shape=(), dtype=float32)
Test Accuracy for batch 32 is:  tf.Tensor(0.60795456, shape=(), dtype=float32)
Test Accuracy for batch 48 is:  tf.Tensor(0.6186224, shape=(), dtype=float32)
Test Accuracy for batch 64 is:  tf.Tensor(0.61538464, shape=(), dtype=float32)
Test Accuracy for batch 80 is:  tf.Tensor(0.61882716, shape=(), dtype=float32)
Test Accuracy for batch 96 is:  tf.Tensor(0.61018044, shape=(), dtype=float32)
Test Accuracy for batch 112 is:  tf.Tensor(0.6050885, shape=(), dtype=float32)
Test Accuracy for batch 128 is:  tf.Tensor(0.5993217, shape=(), dtype=float32)
Test Accuracy for batch 144 is:  tf.Tensor(0.60431033, shape=(), dtype=float32)
Test Accuracy for batch 160 is:  tf.Tensor(0.6086956, shape=(), dtype=float32)
Test Accuracy for batch 176 is:  tf.Tensor(0.6108757, shape=(), dtype=float32)
Test Accuracy for batch 192 is:  tf.Tensor(0.6088083, shape=(), dtype=float32)
Test Accuracy for batch 208 is:  tf.Tensor(0.60855263

KeyboardInterrupt: 